In [1]:
from tshistory.api import timeseries
from BlueOcean import DataAccessApi
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime
tsa = timeseries('http://tst-qdev-ap9.petroineos.local/api')

In [2]:
def bo_get_data(query):
    data = DataAccessApi.GetDataframe(query)
    return data

In [3]:
def clean_country_names(data):
    df = data.copy()
    df['country'] = df['country'].str.lower()
    df['country'] = df['country'].replace(' ', '_', regex=True)
    return df

In [4]:
def safe_register_formula(tsa, name, formula):
    if tsa.exists(name):
        tsa.delete(name)
    tsa.register_formula(name, formula)

In [5]:
ctry_qry = """select distinct country from hive_metastore.dataengineering.oil_refinery_outages_forecast
where pdate = (select max(pdate) from hive_metastore.dataengineering.oil_refinery_outages_forecast)
order by country desc"""
countries = bo_get_data(ctry_qry)

In [6]:
table_names = countries['country'].to_list()
clean_names = clean_country_names(countries)['country'].to_list()

In [8]:
clean_names 

['united_kingdom',
 'turkey',
 'switzerland',
 'sweden',
 'spain',
 'slovakia',
 'portugal',
 'poland',
 'norway',
 'netherlands',
 'italy',
 'ireland',
 'hungary',
 'greece',
 'germany',
 'france',
 'finland',
 'denmark',
 'czech_republic',
 'belgium',
 'austria']

In [22]:
formula = """(add(series "oil.petroineos.germany.unplanned_outage_forecast.kbd.monthly")(series "oil.petroineos.germany.planned_outage_forecast.kbd.monthly"))"""
              
name = f'oil.petroineos.germany.total_outage_forecast.kbd.monthly'
safe_register_formula(tsa, name, formula)

In [14]:
# uploading planned outages
for x, y in zip(table_names, clean_names):
    formula = f"""(blueocean "select FORECAST_DATE as date, UNKNOWN_OUTAGES_KBD as value from hive_metastore.dataengineering.oil_refinery_outages_forecast where pdate = (select max(pdate) from hive_metastore.dataengineering.oil_refinery_outages_forecast) and Country = '{x}' and type = 'Unknown_Planned' order by oil_refinery_outages_forecast.FORECAST_DATE asc")"""
    name = f'oil.petroineos.{y}.planned_outage_forecast.kbd.monthly'
    safe_register_formula(tsa, name, formula)
    print (f'uploaded {x}')

uploaded United Kingdom
uploaded Turkey
uploaded Switzerland
uploaded Sweden
uploaded Spain
uploaded Slovakia
uploaded Portugal
uploaded Poland
uploaded Norway
uploaded Netherlands
uploaded Italy
uploaded Ireland
uploaded Hungary
uploaded Greece
uploaded Germany
uploaded France
uploaded Finland
uploaded Denmark
uploaded Czech Republic
uploaded Belgium
uploaded Austria


In [ ]:
# uploading unplanned outages
for x, y in zip(table_names, clean_names):
    formula = f"""(blueocean "select FORECAST_DATE as date, UNKNOWN_OUTAGES_KBD as value from hive_metastore.dataengineering.oil_refinery_outages_forecast where pdate = (select max(pdate) from hive_metastore.dataengineering.oil_refinery_outages_forecast) and Country = '{x}' and type = 'Unknown_Unplanned' order by oil_refinery_outages_forecast.FORECAST_DATE asc")"""
    name = f'oil.petroineos.{y}.unplanned_outage_forecast.kbd.monthly'
    safe_register_formula(tsa, name, formula)
    print (f'uploaded {x}')

In [23]:
# uploading total outages
for x in clean_names:
    formula = f"""(add(series "oil.petroineos.{x}.unplanned_outage_forecast.kbd.monthly")(series "oil.petroineos.{x}.planned_outage_forecast.kbd.monthly"))"""
    name = f'oil.petroineos.{x}.total_outage_forecast.kbd.monthly'
    safe_register_formula(tsa, name, formula)
    print (f'uploaded {x}')

uploaded united_kingdom
uploaded turkey
uploaded switzerland
uploaded sweden
uploaded spain
uploaded slovakia
uploaded portugal
uploaded poland
uploaded norway
uploaded netherlands
uploaded italy
uploaded ireland
uploaded hungary
uploaded greece
uploaded germany
uploaded france
uploaded finland
uploaded denmark
uploaded czech_republic
uploaded belgium
uploaded austria


In [41]:
# formula = """(blueocean "select FORECAST_DATE as date, UNKNOWN_OUTAGES_KBD as value from hive_metastore.dataengineering.oil_refinery_outages_forecast where pdate = (select max(pdate) from hive_metastore.dataengineering.oil_refinery_outages_forecast) and Country = 'United Kingdom' and type = 'Unknown_Unplanned' order by oil_refinery_outages_forecast.FORECAST_DATE asc")"""
# name = "oil.petroineos.united_kingdom.unplanned_outage_forecast.kbd.monthly"
# safe_register_formula(tsa, name, formula)
# print (formula)

In [24]:
oecd = [f'(series "oil.petroineos.{x}.total_outage_forecast.kbd.monthly")' for x in clean_names]

In [25]:
oecd_total = ' '.join(oecd)

In [26]:
oecd_total

'(series "oil.petroineos.united_kingdom.total_outage_forecast.kbd.monthly") (series "oil.petroineos.turkey.total_outage_forecast.kbd.monthly") (series "oil.petroineos.switzerland.total_outage_forecast.kbd.monthly") (series "oil.petroineos.sweden.total_outage_forecast.kbd.monthly") (series "oil.petroineos.spain.total_outage_forecast.kbd.monthly") (series "oil.petroineos.slovakia.total_outage_forecast.kbd.monthly") (series "oil.petroineos.portugal.total_outage_forecast.kbd.monthly") (series "oil.petroineos.poland.total_outage_forecast.kbd.monthly") (series "oil.petroineos.norway.total_outage_forecast.kbd.monthly") (series "oil.petroineos.netherlands.total_outage_forecast.kbd.monthly") (series "oil.petroineos.italy.total_outage_forecast.kbd.monthly") (series "oil.petroineos.ireland.total_outage_forecast.kbd.monthly") (series "oil.petroineos.hungary.total_outage_forecast.kbd.monthly") (series "oil.petroineos.greece.total_outage_forecast.kbd.monthly") (series "oil.petroineos.germany.total_o